# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096836


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:35,  3.41s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:06,  2.45s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.77s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.31s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  4.68it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.35it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  3.64it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  3.89it/s]

not-logged-in-025360e89444727733a2    0.001059
Jnursssmith5263                       0.000978
Khai_Noct                             0.000365
Quarkgc                               0.000683
jnarayanbvg                           0.000481
deivad                                0.001830
awright5                              0.001865
Hunting_X                             0.002786
rmz4160                               0.000348
ElisabethB                            0.021344
not-logged-in-2f24a5593b112e9f4ae8    0.000267
PuZaw                                 0.000212
jmartinez468                          0.619742
homant                                0.000162
Planetme                              0.014186
acapirala                             0.003807
arad98                                0.005003
aidenr                                0.000513
not-logged-in-3569dcff4dbd0ee90f50    0.002417
Lavadude                              0.016907
not-logged-in-2a217bb163a9b06d92bf    0.001843
ShirleyL     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())